In [ ]:
# default_exp diff_analysis_manager

In [ ]:
from ms_empire.background_distributions import *
from ms_empire.normalization import *
from ms_empire.diff_analysis import *

In [ ]:
#export
#read in peptide datafiles



In [ ]:
#export
import pandas as pd
from itertools import combinations
import numpy as np

def run_pipeline(unnormed_df, labelmap_df, minrep):
    conds = labelmap_df["condition"].unique()
    condpair2diffions = {}
    for condpair in combinations(conds, 2):

        c1_samples = labelmap_df[labelmap_df["condition"]== condpair[0]]
        c2_samples = labelmap_df[labelmap_df["condition"]== condpair[1]]
        df_c1 = unnormed_df.loc[:, c1_samples["sample"]].dropna(thresh=c1_samples.shape[0]-minrep, axis=0)
        df_c2 = unnormed_df.loc[:, c2_samples["sample"]].dropna(thresh=c2_samples.shape[0]-minrep, axis=0)
        
        df_c1_normed = pd.DataFrame(normalize_withincond(df_c1.to_numpy().T).T, index = df_c1.index)
        df_c2_normed = pd.DataFrame(normalize_withincond(df_c2.to_numpy().T).T, index = df_c2.index)

        normed_c1 = ConditionBackgrounds(df_c1_normed)
        normed_c2 = ConditionBackgrounds(df_c2_normed)

        ions_to_check = normed_c1.ion2nonNanvals.keys() & normed_c2.ion2nonNanvals.keys()

        ion2diffDist = {}
        diffIons = []
        for ion in ions_to_check:
            print(f"ion {ion}")
            vals1 = normed_c1.ion2nonNanvals.get(ion)
            vals2 = normed_c2.ion2nonNanvals.get(ion)
            bgdist1 = normed_c1.ion2background.get(ion)
            bgdist2 = normed_c2.ion2background.get(ion)
            diffDist = get_subtracted_bg(ion2diffDist,normed_c1, normed_c2,ion )
            diffIon = DifferentialIon(vals1, vals2, diffDist)
            diffIons.append(diffIon)

        condpair2diffions.update({condpair :diffIon})

In [ ]:
#hide
import pandas as pd
import numpy as np
import uuid

def test_run_pipeline():

    sample2cond_df = pd.DataFrame({'sample' : ['A1', 'A2', 'A3', 'B1', 'B2', 'B3'], 
    'condition' : ['A', 'A', 'A', 'B', 'B', 'B']})
    unnormed_df = generate_random_input(1000, sample2cond_df)
    
    run_pipeline(unnormed_df, sample2cond_df, 3)


def generate_random_input(num_pep,sample2cond_df ):
    pepnames = list(map(lambda _idx : str(uuid.uuid4()), range(num_pep))) #gives uuid strings for each peptide
    randarrays = 10+ 1.5*np.random.randn(num_pep,sample2cond_df.shape[0])
    df_intens = pd.DataFrame(randarrays, columns= sample2cond_df["sample"].tolist())
    df_intens.insert(0, "peptides", pepnames )
    df_intens = df_intens.set_index("peptides")
    return df_intens

test_run_pipeline()


b7be-ca10-4e8e-92e2-5f0844d7e8d9
ion 8fda0c89-6750-45c2-bfd5-77dd650721e5
ion 2c01b826-de79-43eb-b797-a7044f6f3bf5
ion 24284473-4d14-417e-91f4-1893bc12b7ae
ion 464e97d0-515b-47b9-a602-bd702bd8945b
ion 6f3a9966-dcd6-4e43-8c5f-2bb3f1a3bf2d
ion 99952d0f-c969-4fe5-9522-b4561cc94616
ion 42a94e3f-9a15-4685-b705-373870c9a0e2
ion b1c803af-7cb9-403d-b569-5b7536fe9d2e
ion 56292223-b5cc-4552-a9dc-7fe8319616ca
ion c44a021b-47f6-43c1-afea-7a1476b1f413
ion 67cf5819-ef5b-4170-a01e-c3446e067405
ion 1e5a8cc4-655f-4044-b107-1d321d2a716b
ion 84038231-78ed-420c-9ba1-f379a175f8a4
ion 4498db89-45fc-45c0-8df7-8aba17cf5fc0
ion 1ab3531d-a6e4-40a8-bc2b-f5f11371942c
ion aa98143a-4981-4ea7-9c8a-d92534826503
ion 456fdc69-04c5-4c2c-ab00-9a21ca0379f2
ion 21e5e6d6-a5df-4f34-acbd-6dc88267de1e
ion 61a2c09e-fdd3-4616-9b9c-ea73ae81d796
ion eb11bd6f-c9e2-4b74-8f8d-3abe9c59e1bc
ion d1573c01-de6c-418f-a041-b0ecfb043ffd
ion 49819c23-db45-41ea-a0c2-8d4e257ea9e3
ion 03af7441-272c-4346-82f5-b05b1479ca5a
ion c17ac29e-70a2-4a90-b

AttributeError: 'NoneType' object has no attribute 'start_idx'